In [1]:
import peekingduck
assert peekingduck.__version__ == 'v1.1.0' , "Peekingduck is not Updated to the latest version."

In [2]:
from peekingduck.runner import Runner
from peekingduck.pipeline.nodes.input import live
from peekingduck.pipeline.nodes.model import yolo
from peekingduck.pipeline.nodes.draw import bbox, legend, blur_bbox
from peekingduck.pipeline.nodes.dabble import fps
from peekingduck.pipeline.nodes.output import media_writer, screen
from peekingduck.pipeline.nodes.node import AbstractNode
from custom_nodes.debug import printPipe
from custom_nodes.draw import line
from custom_nodes.tracking import tracker, counter

In [4]:
import cv2 as cv
import numpy as np
from urllib.request import urlopen
import os
import datetime
import time
import sys

#change to your ESP32-CAM ip
url="http://192.168.137.86:81/stream"
CAMERA_BUFFRER_SIZE=4096
stream=urlopen(url)
bts=b''
i=0
while True:    
    try:
        bts+=stream.read(CAMERA_BUFFRER_SIZE)
        jpghead=bts.find(b'\xff\xd8')
        jpgend=bts.find(b'\xff\xd9')
        if jpghead>-1 and jpgend>-1:
            jpg=bts[jpghead:jpgend+2]
            bts=bts[jpgend+2:]
            img=cv.imdecode(np.frombuffer(jpg,dtype=np.uint8),cv.IMREAD_UNCHANGED)
            #img=cv.flip(img,0) #>0:垂直翻轉, 0:水平翻轉, <0:垂直水平翻轉            
            #h,w=img.shape[:2]
            #print('影像大小 高:' + str(h) + '寬：' + str(w))
            # img=cv.resize(img,(640,480))
            cv.imshow("a",img)
        else:
            sys.exit(1)
        k=cv.waitKey(1)
    except Exception as e:
        print("Error:" + str(e))
        bts=b''
        stream=urlopen(url)
        sys.exit(1)
        continue
    
    k=cv.waitKey(1)
    # 按a拍照存檔
    if k & 0xFF == ord('a'):
        cv.imwrite(str(i) + ".jpg", img)
        i=i+1
    # 按q離開
    if k & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

In [3]:
# input_node = recorded.Node(input_dir="t1.jpg", mirror_image=True)
# input_node = live.Node(input_source = "http://192.168.162.167:8080/stream.mjpeg", threading= True)
input_node = live.Node(threading = True)
yolo_node = yolo.Node(
    detect_ids =[0,2], # detect human for 0, car for 2
    # yolo_iou_threshold = .2
    )
fps_node = fps.Node(fps_log_display=True)
draw_bbox_node = bbox.Node()
line_node = line.Node()
legend_node = legend.Node()
output_node = screen.Node()
tracker_node = tracker.Node()
counter_node = counter.Node(endpoint = "https://spai-human-counter-backend-api.herokuapp.com/history/")
blur_bbox_node = blur_bbox.Node()
print_node = printPipe.Node(dataInPipe = ['footfall'], skipFrames = 30)

In [4]:
runner = Runner(nodes=[
    input_node, 
    yolo_node, 
    tracker_node,
    counter_node,
    print_node,
    fps_node,
    draw_bbox_node,
    blur_bbox_node,
    line_node,
    legend_node,
    output_node])
runner.run()

0
0
0
0
0


KeyboardInterrupt: 

In [ ]:
import cv2
cv2.destroyAllWindows()

In [ ]:
runner.pipeline.data.keys()